# 1. 기본설정

In [2]:
# 시간 확인
from tqdm import tqdm
import time

# 데이터 전처리 위한 라이브러리
import numpy as np
import numba
import pandas as pd

# Label Encoding을 위한 라이브러리 불러오기
from sklearn.preprocessing import LabelEncoder

# version 확인
import skimage
skimage.__version__

import warnings
warnings.filterwarnings('ignore')

import os

new_directory = 'C:/python'

# 작업 디렉토리 변경
os.chdir(new_directory)

Global Variables

In [3]:
import easydict
args = easydict.EasyDict()

# path 정보
args.default_path = 'Budongsan/new_data/'

args.random_state = 42
args.results = []

# 2. 데이터 불러오기

Load merge data

In [4]:
ori_total_main = pd.read_csv(args.default_path + 'real_final_master_data.csv')

In [5]:
# LabelEncoder 객체 생성 및 적용
label_encoder = LabelEncoder()
ori_total_main['법정동_Encoded'] = label_encoder.fit_transform(ori_total_main['자치구_법정동'])

ori_total_main['법정동_Encoded']

0           0
1           0
2           0
3           0
4           0
         ... 
292460    335
292461    335
292462    335
292463    335
292464    335
Name: 법정동_Encoded, Length: 292465, dtype: int32

In [6]:
median_by_dong = ori_total_main.groupby('법정동')['거래금액(만원)'].median()
ori_total_main['동별거래금액중앙값'] = ori_total_main['법정동'].map(median_by_dong)

# '재건축' 컬럼 생성 및 값 할당
ori_total_main['재건축'] = ori_total_main.apply(lambda row: 1 if (row['재건축 가능 여부'] == 1 and row['거래금액(만원)'] > row['동별거래금액중앙값']) else 0, axis=1)

In [8]:
ori_total_main.loc[ori_total_main['단지명'].str.contains('힐스테이트'), '힐스테이트'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('푸르지오'), '푸르지오'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('더샵'), '더샵'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('e편한세상'), 'e편한세상'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('e-편한세상'), 'e편한세상'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('이편한세상'), 'e편한세상'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('이-편한세상'), 'e편한세상'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('위브'), '위브'] = 1
ori_total_main.loc[ori_total_main['단지명'].str.contains('자이'), '자이'] = 1

In [9]:
ori_total_main['㎡당_평균가격'] = ori_total_main['거래금액(만원)'] / ori_total_main['전용면적(㎡)']

In [10]:
col=['힐스테이트', '푸르지오', '더샵', 'e편한세상', '위브', '자이']

for i in tqdm(col):
    ori_total_main[i]=ori_total_main[i].fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.73it/s]


In [11]:
# 표준편차가 10000 이상으로 차이나는 아파트들만 1로 인식하는 새로운 컬럼 생성

indices = ori_total_main[ori_total_main['단지명'].str.contains('힐스테이트|푸르지오|더샵|e편한세상|e-편한세상|이편한세상|이-편한세상|위브|자이')].index
ori_total_main.loc[indices, '브랜드여부'] = 1

ori_total_main.브랜드여부=ori_total_main.브랜드여부.fillna(0)

데이터 정리

In [12]:
ori_total_main.drop(['Unnamed: 0','시군구', '번지', '본번', '부번', '단지명', '시점', '계약일', '자치구', '도로명_apt', '시', '연도', '월', '도로명주소', '지번주소', '자치구_법정동', '법정동코드', '정류장 ID', '정류소명', '도로명_bus', '역사명', '학교명', '버스정류장_소요시간', '지하철역_소요시간', '공원_소요시간', '학교_소요시간', '법정동', '평단가', '재건축 가능 여부', '동별거래금액중앙값'], axis=1, inplace=True)
ori_total_main.head()

,전용면적(㎡),거래금액(만원),층,건축년도,x,y,총전입,총전출,주민등록인구수,총생활인구수,...,연식,힐스테이트,푸르지오,더샵,e편한세상,위브,자이,브랜드여부,법정동_Encoded,재건축
0,79.97,130000,4,1987.0,127.05593,37.476795,961.0,13256.0,7800.0,837615.719594,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,79.97,117000,2,1987.0,127.05593,37.476795,961.0,13256.0,7800.0,837615.719594,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,79.97,130000,1,1987.0,127.05593,37.476795,961.0,13256.0,7800.0,837615.719594,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,79.97,139500,2,1987.0,127.05593,37.476795,961.0,13256.0,7800.0,826003.757742,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,54.98,107500,5,1987.0,127.05593,37.476795,961.0,13256.0,7800.0,817527.898616,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [13]:
ori_total_main.drop(['x', 'y', '건축년도'], axis=1, inplace=True)
ori_total_main.head()

,전용면적(㎡),거래금액(만원),층,총전입,총전출,주민등록인구수,총생활인구수,버스정류장수,버스정류장_거리,지하철역 수,...,연식,힐스테이트,푸르지오,더샵,e편한세상,위브,자이,브랜드여부,법정동_Encoded,재건축
0,79.97,130000,4,961.0,13256.0,7800.0,837615.719594,80.0,1.42919,2.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
1,79.97,117000,2,961.0,13256.0,7800.0,837615.719594,80.0,1.42919,2.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
2,79.97,130000,1,961.0,13256.0,7800.0,837615.719594,80.0,1.42919,2.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
3,79.97,139500,2,961.0,13256.0,7800.0,826003.757742,80.0,1.42919,2.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
4,54.98,107500,5,961.0,13256.0,7800.0,817527.898616,80.0,1.42919,2.0,...,31.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [14]:
# 정수값을 양수로 변환하는 함수 정의
def make_positive(value):
    if value < 0:
        return -value
    return value

In [15]:
# 특정 열에 있는 모든 값을 양수로 변환
target_column = '층'
ori_total_main[target_column] = ori_total_main[target_column].apply(make_positive)

In [16]:
business_cols = ['건설업', '공공 행정, 국방 및 사회보장 행정', '광업','교육 서비스업', '금융 및 보험업', '농업, 임업 및 어업', '도매 및 소매업', '보건업 및 사회복지 서비스업',
           '부동산업', '사업시설 관리, 사업 지원 및 임대 서비스업', '수도, 하수 및 폐기물 처리, 원료 재생업',
           '숙박 및 음식점업', '예술, 스포츠 및 여가관련 서비스업', '운수 및 창고업',
           '전기, 가스, 증기 및 공기 조절 공급업', '전문, 과학 및 기술 서비스업', '정보통신업', '제조업',
           '협회 및 단체, 수리 및 기타 개인 서비스업']

ori_total_main['사업체'] = ori_total_main[business_cols].apply(lambda cols : cols.sum(),axis = 1)

#컬럼 제거
print(ori_total_main.shape)
ori_total_main.drop(business_cols, axis =1, inplace =True)

print(ori_total_main.shape)

(292465, 68)
(292465, 49)


In [17]:
ori_total_main.columns

Index(['전용면적(㎡)', '거래금액(만원)', '층', '총전입', '총전출', '주민등록인구수', '총생활인구수', '버스정류장수',
       '버스정류장_거리', '지하철역 수', '지하철역_거리', '도로율', '지역내총생산', '월평균소득금액', '기준금리',
       '소비자물가지수', '전세가율', '전월세전환율', '지가지수', '실거래가격지수', '소비심리지수', '음식점 수',
       '카페 수', '편의점 수', '병원 수', '대형마트 수', '백화점 수', '5대범죄(총)', '경제활동인구 (천명)',
       '피보험자수', '공원수', '공원_거리', '학교수', '학교_거리', '부동산', '부동산 매매', '부동산정책',
       '㎡당_평균가격', '연식', '힐스테이트', '푸르지오', '더샵', 'e편한세상', '위브', '자이', '브랜드여부',
       '법정동_Encoded', '재건축', '사업체'],
      dtype='object')

In [18]:
ori_total_main = ori_total_main[ori_total_main['연식']!=-1]

In [19]:
ori_total_main['전용면적_s'] = 0
ori_total_main.loc[ori_total_main['전용면적(㎡)'] < ori_total_main['전용면적(㎡)'].quantile(0.2), '전용면적_s'] = 0
ori_total_main.loc[(ori_total_main['전용면적(㎡)'] >= ori_total_main['전용면적(㎡)'].quantile(0.2)) & (ori_total_main['전용면적(㎡)'] < ori_total_main['전용면적(㎡)'].quantile(0.4)), '전용면적_s'] = 1
ori_total_main.loc[(ori_total_main['전용면적(㎡)'] >= ori_total_main['전용면적(㎡)'].quantile(0.4)) & (ori_total_main['전용면적(㎡)'] < ori_total_main['전용면적(㎡)'].quantile(0.6)), '전용면적_s'] = 2
ori_total_main.loc[(ori_total_main['전용면적(㎡)'] >= ori_total_main['전용면적(㎡)'].quantile(0.6)) & (ori_total_main['전용면적(㎡)'] < ori_total_main['전용면적(㎡)'].quantile(0.8)), '전용면적_s'] = 3
ori_total_main.loc[ori_total_main['전용면적(㎡)'] >= ori_total_main['전용면적(㎡)'].quantile(0.8), '전용면적_s'] = 4

In [20]:
ori_total_main = ori_total_main[['거래금액(만원)', '백화점 수', '공원수', '사업체', '전용면적_s', '학교수', '월평균소득금액', '재건축', '연식', '지하철역 수', '병원 수',
               '기준금리', '층', '피보험자수', '카페 수', '대형마트 수', '부동산정책', '법정동_Encoded', '공원_거리', '부동산 매매', '총전입', '총전출']]

### - 명목형 데이터 Encoding

In [21]:
ori_total_main = ori_total_main.astype({'전용면적_s': str})

In [22]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ori_total_main.columns

Index(['거래금액(만원)', '백화점 수', '공원수', '사업체', '전용면적_s', '학교수', '월평균소득금액', '재건축',
       '연식', '지하철역 수', '병원 수', '기준금리', '층', '피보험자수', '카페 수', '대형마트 수', '부동산정책',
       '법정동_Encoded', '공원_거리', '부동산 매매', '총전입', '총전출'],
      dtype='object')

In [24]:
enc_cols = ['전용면적_s']
normal_cols = list(set(ori_total_main.columns) - set(enc_cols))
normal_cols

['재건축',
 '학교수',
 '피보험자수',
 '공원_거리',
 '백화점 수',
 '사업체',
 '층',
 '부동산 매매',
 '기준금리',
 '총전입',
 '병원 수',
 '부동산정책',
 '지하철역 수',
 '월평균소득금액',
 '공원수',
 '대형마트 수',
 '총전출',
 '법정동_Encoded',
 '카페 수',
 '연식',
 '거래금액(만원)']

In [25]:
enc_cols = list(ori_total_main.select_dtypes(exclude=np.number).columns)
print(f'len_enc_cols: {len(enc_cols)}')
encoder = OneHotEncoder() 
enc_dfs = []

for i, df in enumerate([ori_total_main]):
  if not i:
    _encoded = encoder.fit_transform(df[enc_cols])
  else:
    _encoded = encoder.transform(df[enc_cols])

  _encoded_df = pd.DataFrame(
    _encoded.toarray(), 
    columns = encoder.get_feature_names_out()
  )

  not_enc_cols = list(set(df.columns) - set(enc_cols))
  enc_dfs.append(
    pd.concat(
      [df[not_enc_cols].reset_index(drop=True), _encoded_df.reset_index(drop=True)]
      , axis=1
    ).reset_index(drop=True)
  )

ori_total_main = enc_dfs[0]

print(f'{ori_total_main.isnull().sum().sum()}')
ori_total_main.shape

len_enc_cols: 1
0


(292396, 26)

In [26]:
ori_total_main = ori_total_main.reset_index(drop=True)

### - 수치형 데이터 Scaler

In [27]:
scaling_cols = ['학교수', '부동산정책', '총전출', '사업체', '공원_거리', '월평균소득금액', '지하철역 수', '카페 수', '대형마트 수', '피보험자수',
                 '층', '기준금리', '총전입', '부동산 매매', '백화점 수', '연식', '공원수', '병원 수']
not_scaling_cols = list(set(ori_total_main.columns) - set(scaling_cols))
not_scaling_cols 

['재건축',
 '전용면적_s_0',
 '전용면적_s_1',
 '전용면적_s_3',
 '전용면적_s_4',
 '법정동_Encoded',
 '전용면적_s_2',
 '거래금액(만원)']

In [28]:
from sklearn.preprocessing import RobustScaler

In [29]:
scaler = RobustScaler()

_scaled_tr = scaler.fit_transform(ori_total_main[scaling_cols])
_scaled_te = scaler.transform(ori_total_main[scaling_cols])

In [30]:
scaling_cols = ['학교수', '부동산정책', '총전출', '사업체', '공원_거리', '월평균소득금액', '지하철역 수', '카페 수', '대형마트 수', '피보험자수',
                 '층', '기준금리', '총전입', '부동산 매매', '백화점 수', '연식', '공원수', '병원 수']
scaler = RobustScaler()
scaled_dfs = []

for i, df in enumerate([ori_total_main]):
  if not i:
    _scaled = scaler.fit_transform(df[scaling_cols])
  else:
    _scaled = scaler.transform(df[scaling_cols])

  _scaled_df = pd.DataFrame(
    _scaled, 
    columns = scaling_cols
  )

  not_scaling_cols = list(set(df.columns) - set(scaling_cols))

  scaled_dfs.append(
    pd.concat(
      [df[not_scaling_cols].reset_index(drop=True), _scaled_df.reset_index(drop=True)]
      , axis=1
    ).reset_index(drop=True)
  )

ori_total_main = scaled_dfs[0]

print(f'{ori_total_main.isnull().sum().sum()}')
ori_total_main.shape

0


(292396, 26)

In [31]:
ori_total_main = ori_total_main[['거래금액(만원)', '월평균소득금액', '피보험자수', '전용면적_s_4', '전용면적_s_0', '기준금리', '연식',
       '법정동_Encoded', '재건축', '부동산정책', '카페 수', '층', '전용면적_s_3', '병원 수',
       '학교수', '전용면적_s_1', '사업체', '공원_거리', '부동산 매매', '지하철역 수', '백화점 수',
       '총전출', '총전입', '전용면적_s_2', '공원수', '대형마트 수']]

In [32]:
ori_total_main_s1 = ori_total_main['거래금액(만원)']
ori_total_main_s2 = ori_total_main.drop(['거래금액(만원)'], axis=1)

In [33]:
import joblib  # 모델 저장 및 불러오기
from catboost import CatBoostRegressor

In [34]:
# 저장된 모델 불러오기
loaded_model = CatBoostRegressor()
loaded_model.load_model('CatBoost.cbm')

In [35]:
ori_total_main_s2.columns

Index(['월평균소득금액', '피보험자수', '전용면적_s_4', '전용면적_s_0', '기준금리', '연식', '법정동_Encoded',
       '재건축', '부동산정책', '카페 수', '층', '전용면적_s_3', '병원 수', '학교수', '전용면적_s_1',
       '사업체', '공원_거리', '부동산 매매', '지하철역 수', '백화점 수', '총전출', '총전입', '전용면적_s_2',
       '공원수', '대형마트 수'],
      dtype='object')

In [36]:
# 예측값 계산
new_predictions = loaded_model.predict(ori_total_main_s2)

In [37]:
# 예측값을 새로운 변수로 추가
ori_total_main_s2['예측값'] = new_predictions

In [38]:
ori_total_main_final = pd.concat([ori_total_main_s1, ori_total_main_s2], axis=1)

In [39]:
# 출력 옵션 설정
pd.options.display.float_format = '{:.6f}'.format

ori_total_main_final['재평가정도'] = ori_total_main_final['예측값'] - ori_total_main_final['거래금액(만원)']

In [40]:
ori_total_main_final[ori_total_main_final['재평가정도'] == ori_total_main_final['재평가정도'].min()]

,거래금액(만원),월평균소득금액,피보험자수,전용면적_s_4,전용면적_s_0,기준금리,연식,법정동_Encoded,재건축,부동산정책,...,부동산 매매,지하철역 수,백화점 수,총전출,총전입,전용면적_s_2,공원수,대형마트 수,예측값,재평가정도
281147,1450000,1.336067,4.689128,1.000000,0.000000,1.600000,-1.230769,13,0,0.176270,...,0.205970,-0.333333,0.000000,-0.088004,-0.075435,0.000000,0.000000,0.000000,68161.947973,-1381838.052027


In [41]:
max(ori_total_main_final['재평가정도'])

97022.20524462595

# 3. 검증 & 예측 데이터 파일 생성

In [42]:
ori_total_main_val = pd.read_csv(args.default_path + 'real_final_master_data.csv')

In [43]:
# Label Encoding을 위한 라이브러리 불러오기
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성 및 적용
label_encoder = LabelEncoder()
ori_total_main_val['법정동_Encoded'] = label_encoder.fit_transform(ori_total_main_val['자치구_법정동'])

ori_total_main_val['법정동_Encoded']

0           0
1           0
2           0
3           0
4           0
         ... 
292460    335
292461    335
292462    335
292463    335
292464    335
Name: 법정동_Encoded, Length: 292465, dtype: int32

In [44]:
median_by_dong = ori_total_main_val.groupby('법정동')['거래금액(만원)'].median()
ori_total_main_val['동별거래금액중앙값'] = ori_total_main_val['법정동'].map(median_by_dong)

# '재건축' 컬럼 생성 및 값 할당
ori_total_main_val['재건축'] = ori_total_main_val.apply(lambda row: 1 if (row['재건축 가능 여부'] == 1 and row['거래금액(만원)'] > row['동별거래금액중앙값']) else 0, axis=1)

In [47]:
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('힐스테이트'), '힐스테이트'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('푸르지오'), '푸르지오'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('더샵'), '더샵'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('e편한세상'), 'e편한세상'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('e-편한세상'), 'e편한세상'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('이편한세상'), 'e편한세상'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('이-편한세상'), 'e편한세상'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('위브'), '위브'] = 1
ori_total_main_val.loc[ori_total_main_val['단지명'].str.contains('자이'), '자이'] = 1

In [48]:
ori_total_main_val['㎡당_평균가격'] = ori_total_main_val['거래금액(만원)'] / ori_total_main_val['전용면적(㎡)']

In [49]:
col=['힐스테이트', '푸르지오', '더샵', 'e편한세상', '위브', '자이']

for i in tqdm(col):
    ori_total_main_val[i]=ori_total_main_val[i].fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.01it/s]


In [50]:
# 표준편차가 10000 이상으로 차이나는 아파트들만 1로 인식하는 새로운 컬럼 생성

indices = ori_total_main_val[ori_total_main_val['단지명'].str.contains('힐스테이트|푸르지오|더샵|e편한세상|e-편한세상|이편한세상|이-편한세상|위브|자이')].index
ori_total_main_val.loc[indices, '브랜드여부'] = 1

ori_total_main_val.브랜드여부=ori_total_main_val.브랜드여부.fillna(0)

데이터 정리

In [51]:
# 정수값을 양수로 변환하는 함수 정의
def make_positive(value):
    if value < 0:
        return -value
    return value

In [52]:
# 특정 열에 있는 모든 값을 양수로 변환
target_column = '층'
ori_total_main_val[target_column] = ori_total_main_val[target_column].apply(make_positive)

In [53]:
business_cols = ['건설업', '공공 행정, 국방 및 사회보장 행정', '광업','교육 서비스업', '금융 및 보험업', '농업, 임업 및 어업', '도매 및 소매업', '보건업 및 사회복지 서비스업',
           '부동산업', '사업시설 관리, 사업 지원 및 임대 서비스업', '수도, 하수 및 폐기물 처리, 원료 재생업',
           '숙박 및 음식점업', '예술, 스포츠 및 여가관련 서비스업', '운수 및 창고업',
           '전기, 가스, 증기 및 공기 조절 공급업', '전문, 과학 및 기술 서비스업', '정보통신업', '제조업',
           '협회 및 단체, 수리 및 기타 개인 서비스업']

ori_total_main_val['사업체'] = ori_total_main_val[business_cols].apply(lambda cols : cols.sum(),axis = 1)

#컬럼 제거
print(ori_total_main_val.shape)
ori_total_main_val.drop(business_cols, axis =1, inplace =True)

print(ori_total_main_val.shape)

(292465, 101)
(292465, 82)


In [54]:
ori_total_main_val.columns

Index(['Unnamed: 0', '시군구', '번지', '본번', '부번', '단지명', '전용면적(㎡)', '시점', '계약일',
       '거래금액(만원)', '층', '건축년도', '도로명_apt', '시', '자치구', '법정동', '연도', '월',
       '도로명주소', '지번주소', 'x', 'y', '자치구_법정동', '법정동코드', '총전입', '총전출', '주민등록인구수',
       '총생활인구수', '정류장 ID', '정류소명', '도로명_bus', '버스정류장수', '버스정류장_거리',
       '버스정류장_소요시간', '역사명', '지하철역 수', '지하철역_거리', '지하철역_소요시간', '도로율', '지역내총생산',
       '월평균소득금액', '기준금리', '소비자물가지수', '전세가율', '전월세전환율', '지가지수', '실거래가격지수',
       '소비심리지수', '음식점 수', '카페 수', '편의점 수', '병원 수', '대형마트 수', '백화점 수',
       '5대범죄(총)', '경제활동인구 (천명)', '피보험자수', '공원수', '공원_거리', '공원_소요시간', '학교명',
       '학교수', '학교_거리', '학교_소요시간', '부동산', '부동산 매매', '부동산정책', '평단가', '㎡당_평균가격',
       '연식', '재건축 가능 여부', '힐스테이트', '푸르지오', '더샵', 'e편한세상', '위브', '자이', '브랜드여부',
       '법정동_Encoded', '동별거래금액중앙값', '재건축', '사업체'],
      dtype='object')

In [55]:
ori_total_main_final.columns

Index(['거래금액(만원)', '월평균소득금액', '피보험자수', '전용면적_s_4', '전용면적_s_0', '기준금리', '연식',
       '법정동_Encoded', '재건축', '부동산정책', '카페 수', '층', '전용면적_s_3', '병원 수', '학교수',
       '전용면적_s_1', '사업체', '공원_거리', '부동산 매매', '지하철역 수', '백화점 수', '총전출', '총전입',
       '전용면적_s_2', '공원수', '대형마트 수', '예측값', '재평가정도'],
      dtype='object')

In [56]:
ori_total_main_val = ori_total_main_val[ori_total_main_val['연식']!=-1]

In [57]:
ori_total_main_val['전용면적_s'] = 0
ori_total_main_val.loc[ori_total_main_val['전용면적(㎡)'] < ori_total_main_val['전용면적(㎡)'].quantile(0.2), '전용면적_s'] = 0
ori_total_main_val.loc[(ori_total_main_val['전용면적(㎡)'] >= ori_total_main_val['전용면적(㎡)'].quantile(0.2)) & (ori_total_main_val['전용면적(㎡)'] < ori_total_main_val['전용면적(㎡)'].quantile(0.4)), '전용면적_s'] = 1
ori_total_main_val.loc[(ori_total_main_val['전용면적(㎡)'] >= ori_total_main_val['전용면적(㎡)'].quantile(0.4)) & (ori_total_main_val['전용면적(㎡)'] < ori_total_main_val['전용면적(㎡)'].quantile(0.6)), '전용면적_s'] = 2
ori_total_main_val.loc[(ori_total_main_val['전용면적(㎡)'] >= ori_total_main_val['전용면적(㎡)'].quantile(0.6)) & (ori_total_main_val['전용면적(㎡)'] < ori_total_main_val['전용면적(㎡)'].quantile(0.8)), '전용면적_s'] = 3
ori_total_main_val.loc[ori_total_main_val['전용면적(㎡)'] >= ori_total_main_val['전용면적(㎡)'].quantile(0.8), '전용면적_s'] = 4

In [58]:
ori_total_main_val = ori_total_main_val.reset_index(drop=True)

In [59]:
ori_total_main_final_s2 = ori_total_main_final.drop(['법정동_Encoded'], axis=1)

In [60]:
ori_total_main_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292396 entries, 0 to 292395
Data columns (total 83 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   292396 non-null  int64  
 1   시군구          292396 non-null  object 
 2   번지           292396 non-null  object 
 3   본번           292396 non-null  float64
 4   부번           292396 non-null  float64
 5   단지명          292396 non-null  object 
 6   전용면적(㎡)      292396 non-null  float64
 7   시점           292396 non-null  object 
 8   계약일          292396 non-null  int64  
 9   거래금액(만원)     292396 non-null  int64  
 10  층            292396 non-null  int64  
 11  건축년도         292396 non-null  float64
 12  도로명_apt      292396 non-null  object 
 13  시            292396 non-null  object 
 14  자치구          292396 non-null  object 
 15  법정동          292396 non-null  object 
 16  연도           292396 non-null  int64  
 17  월            292396 non-null  int64  
 18  도로명주소        292396 non-

In [61]:
ori_total_main_final_s2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292396 entries, 0 to 292395
Data columns (total 27 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   거래금액(만원)  292396 non-null  int64  
 1   월평균소득금액   292396 non-null  float64
 2   피보험자수     292396 non-null  float64
 3   전용면적_s_4  292396 non-null  float64
 4   전용면적_s_0  292396 non-null  float64
 5   기준금리      292396 non-null  float64
 6   연식        292396 non-null  float64
 7   재건축       292396 non-null  int64  
 8   부동산정책     292396 non-null  float64
 9   카페 수      292396 non-null  float64
 10  층         292396 non-null  float64
 11  전용면적_s_3  292396 non-null  float64
 12  병원 수      292396 non-null  float64
 13  학교수       292396 non-null  float64
 14  전용면적_s_1  292396 non-null  float64
 15  사업체       292396 non-null  float64
 16  공원_거리     292396 non-null  float64
 17  부동산 매매    292396 non-null  float64
 18  지하철역 수    292396 non-null  float64
 19  백화점 수     292396 non-null  float64
 20  총전출 

In [62]:
ori_total_main_final_s3 = pd.concat([ori_total_main_val, ori_total_main_final_s2[['예측값','재평가정도']]], axis=1)

In [63]:
ori_total_main_final_s3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 292396 entries, 0 to 292395
Data columns (total 85 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Unnamed: 0   292396 non-null  int64  
 1   시군구          292396 non-null  object 
 2   번지           292396 non-null  object 
 3   본번           292396 non-null  float64
 4   부번           292396 non-null  float64
 5   단지명          292396 non-null  object 
 6   전용면적(㎡)      292396 non-null  float64
 7   시점           292396 non-null  object 
 8   계약일          292396 non-null  int64  
 9   거래금액(만원)     292396 non-null  int64  
 10  층            292396 non-null  int64  
 11  건축년도         292396 non-null  float64
 12  도로명_apt      292396 non-null  object 
 13  시            292396 non-null  object 
 14  자치구          292396 non-null  object 
 15  법정동          292396 non-null  object 
 16  연도           292396 non-null  int64  
 17  월            292396 non-null  int64  
 18  도로명주소        292396 non-

In [64]:
ori_total_main_final_s3[ori_total_main_final_s3['재평가정도'] == ori_total_main_final_s3['재평가정도'].max()]

,Unnamed: 0,시군구,번지,본번,부번,단지명,전용면적(㎡),시점,계약일,거래금액(만원),...,위브,자이,브랜드여부,법정동_Encoded,동별거래금액중앙값,재건축,사업체,전용면적_s,예측값,재평가정도
155795,155844,서울특별시 중랑구 신내동,616-1,616.000000,1.000000,월드크라운,110.470000,2019-12-01,24,29000,...,0.000000,0.000000,0,334,43800.000000,0,4811.000000,4,126022.205245,97022.205245


In [65]:
ori_total_main_final_s3 = ori_total_main_final_s3.drop(['Unnamed: 0'], axis=1)

In [66]:
ori_total_main_final_s3.to_csv('apt_Prediction_Evaluation.csv', index=False)